In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

C:\Users\narendran.thesma\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from bayes_opt import BayesianOptimization

In [3]:
# data_1 = pd.read_csv("Adidas GPMT/GPMT Project/15th Oct 35 CSV.csv", encoding='latin-1')
# data_2 = pd.read_csv("Adidas GPMT/GPMT Project/15th Oct 37 CSV.csv", encoding='latin-1')
# data_3 = pd.read_csv("Adidas GPMT/GPMT Project/15th Oct 38 CSV.csv", encoding='latin-1')
# data_4 = pd.read_csv("Adidas GPMT/GPMT Project/15th Oct 39 CSV.csv", encoding='latin-1')
# data = data_1.append(data_2, ignore_index=True)
# data = data.append(data_3, ignore_index=True)
# data = data.append(data_4, ignore_index=True)
# data.head()

In [4]:
data = pd.read_csv("Japan_Not_Encoded.csv", encoding='utf-8')
data.head()

,COMPANY,COUNTRY,DISTRIBUTOR,ARTICLE ID,MODEL NUMBER,ARTICLE NAME,SUBBRAND,SPORTS CATEGORY,PRODUCT DIVISION,PRODUCT GROUP,...,CURRENT PRICE IN EUR,INITIAL PRICE IN SELECTED CURRENCY,CURRENT PRICE IN SELECTED CURRENCY,SELECTED CURRENCY,PRODUCT INTRODUCTION DATE,DISCOUNTED SINCE,PRODUCT EXIT DATE,PRODUCT DESCRIPTION,PRODUCT URL,IMAGE-SERVER URL
0,Nike,Japan,Own eCom,BG0387-001,BG0387,ナイキ コア ハーフ KV ゴルフバッグ,Performance,Golf,Accessories,Bags,...,99.144,116.640,99.144,EUR,2/22/2016,12/26/2016,1/2/2017,整理しやすいゴルフバッグ。快適な持ち運び。 ナイキ コア ハーフ KV ゴルフバッグは、専用...,http://store.nike.com/jp/ja_jp/pd/%25E3%2583%2...,http://usporamap287.am.adsint.biz/zoomimages/1...
1,Nike,Japan,Own eCom,839240-001,839240,ナイキ コルテッツ QS キッズシューズ,Sport Inspired,Lifestyle,Footwear,Sport Inspired Footwear,...,62.820,82.620,62.820,EUR,2/22/2016,4/5/2016,7/26/2016,高級感のあるレトロスタイル ナイキ コルテッツ QS キッズシューズは、上質なレザーのア...,http://store.nike.com/jp/ja_jp/pd/%25E3%2583%2...,http://usporamap287.am.adsint.biz/zoomimages/1...
2,Nike,Japan,Own eCom,GL0783-101,GL0783,ナイキ レジン スピード レッド ゴルフボール,Performance,Golf,Accessories,Sport Equipment,...,31.104,31.104,31.104,EUR,2/15/2016,Not discounted yet,8/29/2016,高初速でより遠くへ ナイキ レジン スピード レッド ゴルフボールは、更にソフトになった新開...,http://store.nike.com/jp/ja_jp/pd/%25E3%2583%2...,http://usporamap287.am.adsint.biz/zoomimages/1...
3,Nike,Japan,Own eCom,GL0781-101,GL0781,ナイキ レジン ツアー ブラック ゴルフボール,Performance,Golf,Accessories,Sport Equipment,...,28.764,58.320,28.764,EUR,2/15/2016,2/27/2017,6/27/2017,低スピンでより遠くへ ナイキ レジン ツアー ブラック ゴルフボールは、更にソフトになったR...,http://store.nike.com/jp/ja_jp/pd/%25E3%2583%2...,http://usporamap287.am.adsint.biz/zoomimages/1...
4,Nike,Japan,Own eCom,AC3877-005,AC3877,ナイキ ATG スピード ジャンプ ロープ,Performance,Training,Accessories,Sport Equipment,...,28.188,28.188,28.188,EUR,2/15/2016,Not discounted yet,7/19/2016,軽く、速く、カスタマイズも可能。 ナイキ ATG スピード ジャンプ ロープは、滑りにくいボ...,http://store.nike.com/jp/ja_jp/pd/%25E3%2583%2...,http://usporamap287.am.adsint.biz/zoomimages/1...


In [5]:
data.COUNTRY.unique()

array(['Japan'], dtype=object)

In [6]:
data = data.loc[data["COUNTRY"].isin(['Japan'])]

In [7]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [40]:
data_req = data.loc[:,["COMPANY", "COUNTRY", "ARTICLE NAME","SUBBRAND", "PRODUCT DESCRIPTION", "PRODUCT URL"]]

In [41]:
data_req = data_req.drop_duplicates()

In [42]:
data_req["description"]  = data_req["COMPANY"] +" " + data_req["ARTICLE NAME"] +" " + data_req["PRODUCT DESCRIPTION"]

In [43]:
data_req.columns

Index(['COMPANY', 'COUNTRY', 'ARTICLE NAME', 'SUBBRAND', 'PRODUCT DESCRIPTION',
       'PRODUCT URL', 'description'],
      dtype='object')

In [44]:
data_req = data_req.drop(["COUNTRY", "ARTICLE NAME", "PRODUCT DESCRIPTION", "PRODUCT URL"], axis = 1)
data_req.columns = ["COMPANY", "subBrand", "description"]

In [45]:
data_req.description = data_req.description.fillna("unknown")

In [46]:
data_req["subBrand"] = data_req["subBrand"].str.lower()
data_req.description = data_req.description.str.lower()

In [47]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(data_req["subBrand"].fillna("unknown"))

In [48]:
X = ["description", "COMPANY"]

In [49]:
data_req["subBrand"].value_counts()

performance       33771
sport inspired    27764
Name: subBrand, dtype: int64

In [50]:
xtrain, xvalid, ytrain, yvalid = train_test_split(data_req[X], y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [51]:
(xtrain)

,description,COMPANY
13161,nike ナイキ エピック ラックス ウィメンズ ランニングタンクトップ ナイキ エピック ...,Nike
41518,unknown,adidas
24720,"adidas ピュアブーストエックス 【pure boost x tr】 , サイズ： 素材...",adidas
32499,puma twv ピステトップ puma twv ピステトップ｜フォームストライプを基調とし...,Puma
44284,reebok 【2017春夏】ufan デュアルブレンド ショートスリーブ グラフィックtシ...,Reebok
10601,nike ナイキ クラシック クーリング ウィメンズ ミディアム インパクト スポーツブラ ...,Nike
13190,nike ナイキ テイルウィンド ラン ディビジョン ウィメンズ ランニングタンクトップ ナ...,Nike
12248,nike ナイキ エア マックス 90 se レザー ベビーシューズ ナイキ エア マックス...,Nike
29506,puma トート pt マチ付き、大容量のトートバックです。ハンドバックとして使用できるのは...,Puma
30952,puma red bull racing ディスク 1991年、世界初のシューレースの無いラ...,Puma


In [52]:
japanese_sw = pd.read_csv("Japanese_SW.csv", encoding="utf-8")

In [25]:
#japanese_sw.STOP_WORD.tolist()

In [26]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = japanese_sw.STOP_WORD.tolist())

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [21]:
# Fitting simple logistic regression
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.133 


In [80]:
predict_label = [np.argmax(i) for i in predictions]

In [81]:
predict_label = lbl_enc.inverse_transform(predict_label)

C:\Users\narendran.thesma\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [82]:
yvalid_label = lbl_enc.inverse_transform(yvalid)

C:\Users\narendran.thesma\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [83]:
yvalid_label

array(['performance', 'sport inspired', 'sport inspired', ...,
       'performance', 'performance', 'performance'], dtype=object)

In [84]:
test_df = pd.DataFrame({'description' : xvalid, 'actuals' : yvalid_label, 'predictions' : predict_label})

In [85]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [86]:
f1_score(test_df.actuals, test_df.predictions,average="macro")

0.9534620748244076

In [87]:
accuracy_score(test_df.actuals, test_df.predictions)

0.9568909209666885

In [19]:
xtrain

array(['nike japan ナイキ エピック ラックス ウィメンズ ランニングタンクトップ ナイキ エピック ラックス ウィメンズ ランニングタンクトップは、速乾性に優れたコンプレッション素材を使用。ランニング中もさらりと快適でサポート力に優れた着心地が持続します。 https://www.nike.com/jp/t/%e3%83%8a%e3%82%a4%e3%82%ad-%e3%82%a8%e3%83%94%e3%83%83%e3%82%af-%e3%83%a9%e3%83%83%e3%82%af%e3%82%b9-%e3%83%a9%e3%83%b3%e3%83%8b%e3%83%b3%e3%82%b0%e3%82%bf%e3%83%b3%e3%82%af%e3%83%88%e3%83%83%e3%83%97-j8wtpw',
       'unknown',
       'adidas japan ピュアブーストエックス 【pure boost x tr】 , サイズ： 素材： 生産国： https://shop.adidas.jp/products/aq1971/',
       ...,
       'nike japan ナイキラボ ベナッシ ラックス ブーツ ウィメンズブーツ 斬新なイメージチェンジ ナイキラボ ベナッシ ラックス ウィメンズブーツは、すっきりしたデザインとプレミアム感あふれるディテールで、クラシックなスライドをリメイク。特徴的な前足部のストラップを受け継ぐこのハイカットのブーツは、通気孔付きのswooshロゴを配し、履く人の視点で考え抜かれたデザインです。 特長 プレミアムレザーのアッパーとインソールで、ワンランク上のスタイルと履き心地を実現 インジェクション成型のphylonソールが柔らかい軽量クッショニングを提供 ヘリンボーンパターンのアウトソールが耐久性に優れたトラクションを発揮 後部のジッパーとレザーのプルタブ 原産国： 中国 http://store.nike.com/jp/ja_jp/pd/%25e3%2583%258a%25e3%2582%25a4%25e3%2582%25ad%25e3%2583%25a9%25e3%2583%259c-%25e3%2583%2599%25e3%2583%258a%25

In [28]:
# Count vectorizer
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = japanese_sw.STOP_WORD.tolist())

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

In [64]:
# Fitting a simple Logistic Regression on Counts
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.094 


In [65]:
# Fitting a simple Naive Bayes on TFIDF
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.194 


In [18]:
# Fitting a simple Naive Bayes on Counts
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 2.597 


In [36]:
# # Apply SVD, I chose 120 components. 120-200 components are good enough for SVM model.
# svd = decomposition.TruncatedSVD(n_components=120)
# svd.fit(xtrain_tfv)
# xtrain_svd = svd.transform(xtrain_tfv)
# xvalid_svd = svd.transform(xvalid_tfv)

# # Scale the data obtained from SVD. Renaming variable to reuse without scaling.
# scl = preprocessing.StandardScaler()
# scl.fit(xtrain_svd)
# xtrain_svd_scl = scl.transform(xtrain_svd)
# xvalid_svd_scl = scl.transform(xvalid_svd)

In [ ]:
# Fitting a simple SVM
# clf = SVC(C=1.0, probability=True) # since we need probabilities
# clf.fit(xtrain_svd_scl, ytrain)
# predictions = clf.predict_proba(xvalid_svd_scl)

# print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

In [79]:
# Fitting a simple xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_tfv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.129 


In [17]:
# Fitting a simple xgboost on count
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.144 


In [ ]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

In [ ]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

Grid search

In [21]:
mll_scorer = metrics.make_scorer(multiclass_logloss, greater_is_better=False, needs_proba=True)

In [ ]:
# Initialize SVD
svd = TruncatedSVD()
    
# Initialize the standard scaler 
scl = preprocessing.StandardScaler()

# We will use logistic regression here..
lr_model = LogisticRegression()

# Create the pipeline 
clf = pipeline.Pipeline([('svd', svd),
                         ('scl', scl),
                         ('lr', lr_model)])

In [ ]:
param_grid = {'svd__n_components' : [120, 180],
              'lr__C': [0.1, 1.0, 10], 
              'lr__penalty': ['l1', 'l2']}

In [ ]:
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)  # we can use the full data here but im only using xtrain
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
nb_model = MultinomialNB()

# Create the pipeline 
clf = pipeline.Pipeline([('nb', nb_model)])

# parameter grid
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)  # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 6 candidates, totalling 12 fits


In [67]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('D:/GPMT/fasttext/cc.de.300.vec', encoding="utf-8")
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2000001it [06:46, 4918.69it/s]


Found 2000000 word vectors.


In [56]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s)

In [58]:
chk = [tokenize(x) for x in xtrain.description]

In [62]:
len(xtrain.description)

55381

In [63]:
import tinysegmenter
segmenter = tinysegmenter.TinySegmenter()
tokenized_text = [segmenter.tokenize(x) for x in chk ]


In [54]:
x = tokenized_text.index

In [64]:
tokenized_text[2]

['adidas',
 ' ',
 'ピュアブーストエックス',
 ' ',
 '【',
 'pure',
 ' ',
 'boost',
 ' ',
 'x',
 ' ',
 'tr',
 '】',
 '  ',
 ',',
 '  ',
 'サイズ',
 '：',
 ' ',
 '素材',
 '：',
 ' ',
 '生産国',
 '：']

In [77]:
word_model.wv.vocab

{'nike': <gensim.models.keyedvectors.Vocab at 0x29ff4716ba8>,
 ' ': <gensim.models.keyedvectors.Vocab at 0x29ff4716550>,
 'ナイキ': <gensim.models.keyedvectors.Vocab at 0x29ff4716b70>,
 'エピック': <gensim.models.keyedvectors.Vocab at 0x29ff4cf0518>,
 'ラックス': <gensim.models.keyedvectors.Vocab at 0x29ff4d2b630>,
 'ウィメンズ': <gensim.models.keyedvectors.Vocab at 0x29ff4d9b6d8>,
 'ランニングタンクトップ': <gensim.models.keyedvectors.Vocab at 0x29ff4dc2208>,
 'は': <gensim.models.keyedvectors.Vocab at 0x29ff4dc2240>,
 '、': <gensim.models.keyedvectors.Vocab at 0x29ff4dc2278>,
 '速乾': <gensim.models.keyedvectors.Vocab at 0x29ff4dc22b0>,
 '性': <gensim.models.keyedvectors.Vocab at 0x29ff4dc22e8>,
 'に': <gensim.models.keyedvectors.Vocab at 0x29ff4dc2320>,
 '優れ': <gensim.models.keyedvectors.Vocab at 0x29ff4dc2358>,
 'た': <gensim.models.keyedvectors.Vocab at 0x29ff4dc2390>,
 'コンプレッション': <gensim.models.keyedvectors.Vocab at 0x29ff4dc23c8>,
 '素材': <gensim.models.keyedvectors.Vocab at 0x29ff4dc2400>,
 'を': <gensim.models.

In [65]:
import gensim
word_model = gensim.models.Word2Vec(tokenized_text, size=300, min_count=1, window=5, iter=1)

C:\Users\narendran.thesma\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [73]:
word

'Fukufuji'

In [74]:
embeddings_index["Fukufuji"]

array([ 0.0311, -0.0149,  0.0287,  0.0131,  0.0231,  0.0147,  0.0548,
        0.0144,  0.0334,  0.0166,  0.0305, -0.0256, -0.0456,  0.006 ,
        0.0226,  0.0174,  0.0124, -0.0156,  0.0132,  0.0064, -0.0703,
        0.0235,  0.0422, -0.0024,  0.0089, -0.0326,  0.0558,  0.006 ,
       -0.0373, -0.0012, -0.0064,  0.0474,  0.0046, -0.0124, -0.0053,
        0.0596, -0.0042,  0.0137,  0.0841,  0.0027, -0.022 , -0.0055,
        0.024 ,  0.0104,  0.0416, -0.0297, -0.0301,  0.0503, -0.0068,
       -0.0196, -0.0387,  0.0245, -0.0059,  0.0124,  0.0593, -0.0142,
        0.0509, -0.0353, -0.0475,  0.0294,  0.0126,  0.0362,  0.0158,
       -0.0093,  0.0341, -0.0453,  0.0366,  0.0164, -0.0266,  0.0055,
       -0.0271,  0.0263, -0.0052,  0.0094,  0.0194, -0.0077,  0.0222,
       -0.004 , -0.0099,  0.0429,  0.0386, -0.0344,  0.0285, -0.0152,
        0.0144,  0.013 , -0.0067, -0.0236,  0.0183, -0.0206, -0.0582,
        0.0047,  0.0116,  0.0107,  0.0167, -0.0571,  0.0187, -0.0025,
        0.0142, -0.0

In [24]:
# this function creates a normalized vector for the whole sentence
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [25]:
# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xvalid_glove = [sent2vec(x) for x in tqdm(xvalid)]

  0%|                                                                                | 2/6154 [00:00<00:06, 998.88it/s]


In [26]:
xtrain_glove = np.asarray(xtrain_glove)
xvalid_glove = np.asarray(xvalid_glove)

In [24]:
# Fitting a simple xgboost on glove features
clf = xgb.XGBClassifier(silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.330 


In [23]:
# Fitting a simple xgboost on glove features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, learning_rate=0.1, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.117 


In [24]:
def XGB_CV(
          max_depth,
          gamma,
          min_child_weight,
          max_delta_step,
          subsample,
          colsample_bytree
         ):

    global AUCbest
    global ITERbest

#
# Define all XGboost parameters
#

    paramt = {
              'booster' : 'gbtree',
              'max_depth' : int(max_depth),
              'gamma' : gamma,
              'eta' : 0.1,
              'objective' : 'binary:logistic',
              'nthread' : 4,
              'silent' : True,
              'eval_metric': 'auc',
              'subsample' : max(min(subsample, 1), 0),
              'colsample_bytree' : max(min(colsample_bytree, 1), 0),
              'min_child_weight' : min_child_weight,
              'max_delta_step' : int(max_delta_step),
              'seed' : 1001
              }

    folds = 5
    cv_score = 0

    print("\n Search parameters (%d-fold validation):\n %s" % (folds, paramt), file=log_file )
    log_file.flush()

    xgbc = xgb.cv(
                    paramt,
                    dtrain,
                    num_boost_round = 20000,
                    stratified = True,
                    nfold = folds,
#                    verbose_eval = 10,
                    early_stopping_rounds = 100,
                    metrics = 'auc',
                    show_stdv = True
               )

# This line would have been on top of this section
#    with capture() as result:

# After xgb.cv is done, this section puts its output into log file. Train and validation scores 
# are also extracted in this section. Note the "diff" part in the printout below, which is the 
# difference between the two scores. Large diff values may indicate that a particular set of 
# parameters is overfitting, especially if you check the CV portion of it in the log file and find 
# out that train scores were improving much faster than validation scores.

#    print('', file=log_file)
#    for line in result[1]:
#        print(line, file=log_file)
#    log_file.flush()

    val_score = xgbc['test-auc-mean'].iloc[-1]
    train_score = xgbc['train-auc-mean'].iloc[-1]
    print(' Stopped after %d iterations with train-auc = %f val-auc = %f ( diff = %f ) train-gini = %f val-gini = %f' % ( len(xgbc), train_score, val_score, (train_score - val_score), (train_score*2-1),
(val_score*2-1)) )
    if ( val_score > AUCbest ):
        AUCbest = val_score
        ITERbest = len(xgbc)

    return (val_score*2) - 1

In [25]:
XGB_BO = BayesianOptimization(XGB_CV, {
                                     'max_depth': (2, 12),
                                     'gamma': (0.001, 10.0),
                                     'min_child_weight': (0, 20),
                                     'max_delta_step': (0, 10),
                                     'subsample': (0.4, 1.0),
                                     'colsample_bytree' :(0.4, 1.0)
                                    })

In [26]:
XGB_BO.explore({
              'max_depth':            [3, 8],
              'gamma':                [0.5, 8],
              'min_child_weight':     [0.2, 12],
              'max_delta_step':       [1, 2],
              'subsample':            [0.6, 0.8],
              'colsample_bytree':     [0.6, 0.8],
              })

In [27]:
import warnings
from sklearn.metrics import log_loss, matthews_corrcoef, roc_auc_score
import contextlib

In [28]:
log_file = open('Porto-AUC-5fold-XGB-run-01-v1-full.log', 'a')
AUCbest = -1.
ITERbest = 0

In [29]:
dtrain = xgb.DMatrix(xtrain_glove, label =ytrain)

In [ ]:
print('-'*130)
print('-'*130, file=log_file)
log_file.flush()

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    XGB_BO.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)

----------------------------------------------------------------------------------------------------------------------------------
Initialization
----------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_delta_step |   max_depth |   min_child_weight |   subsample | 
 Stopped after 3436 iterations with train-auc = 0.999556 val-auc = 0.989799 ( diff = 0.009757 ) train-gini = 0.999112 val-gini = 0.979598
    1 | 156m38s |    0.97960 |             0.6000 |    0.5000 |           1.0000 |      3.0000 |             0.2000 |      0.6000 | 


Deep learning

In [25]:
# scale the data before any neural net:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [137]:
# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

In [27]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
chk = le.fit_transform(data["SPORTS CATEGORY"])

In [50]:
# create a simple 3 layer sequential neural net
from keras import optimizers

model = Sequential()

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(200, activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(150, activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(100, activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(50, activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(2))
model.add(Activation('softmax'))

# compile the model
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

In [51]:
model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=64, 
          epochs=50, verbose=1, 
          validation_data=(xvalid_glove_scl, yvalid_enc))

Train on 55381 samples, validate on 6154 samples
Epoch 1/50
53440/55381 [===========================>..] - ETA: 42:39 - loss: 0.80 - ETA: 8:40 - loss: 0.9679 - ETA: 6:17 - loss: 0.959 - ETA: 4:59 - loss: 0.999 - ETA: 3:48 - loss: 0.995 - ETA: 3:05 - loss: 1.006 - ETA: 2:37 - loss: 0.993 - ETA: 2:23 - loss: 0.977 - ETA: 2:01 - loss: 0.962 - ETA: 1:53 - loss: 0.964 - ETA: 1:44 - loss: 0.965 - ETA: 1:35 - loss: 0.954 - ETA: 1:28 - loss: 0.945 - ETA: 1:20 - loss: 0.940 - ETA: 1:17 - loss: 0.933 - ETA: 1:16 - loss: 0.929 - ETA: 1:13 - loss: 0.930 - ETA: 1:08 - loss: 0.927 - ETA: 1:06 - loss: 0.924 - ETA: 1:04 - loss: 0.917 - ETA: 1:01 - loss: 0.917 - ETA: 59s - loss: 0.910 - ETA: 55s - loss: 0.90 - ETA: 53s - loss: 0.90 - ETA: 51s - loss: 0.90 - ETA: 49s - loss: 0.89 - ETA: 47s - loss: 0.89 - ETA: 46s - loss: 0.88 - ETA: 44s - loss: 0.88 - ETA: 44s - loss: 0.88 - ETA: 43s - loss: 0.88 - ETA: 42s - loss: 0.88 - ETA: 42s - loss: 0.87 - ETA: 41s - loss: 0.87 - ETA: 40s - loss: 0.87 - ETA: 39s 

55104/55381 [============================>.] - ETA: 13s - loss: 0.60 - ETA: 16s - loss: 0.52 - ETA: 16s - loss: 0.50 - ETA: 15s - loss: 0.51 - ETA: 16s - loss: 0.52 - ETA: 16s - loss: 0.51 - ETA: 16s - loss: 0.52 - ETA: 16s - loss: 0.52 - ETA: 16s - loss: 0.52 - ETA: 16s - loss: 0.52 - ETA: 16s - loss: 0.52 - ETA: 17s - loss: 0.53 - ETA: 18s - loss: 0.53 - ETA: 18s - loss: 0.53 - ETA: 18s - loss: 0.53 - ETA: 18s - loss: 0.53 - ETA: 18s - loss: 0.53 - ETA: 18s - loss: 0.53 - ETA: 18s - loss: 0.53 - ETA: 18s - loss: 0.53 - ETA: 18s - loss: 0.53 - ETA: 18s - loss: 0.53 - ETA: 18s - loss: 0.53 - ETA: 18s - loss: 0.53 - ETA: 18s - loss: 0.53 - ETA: 17s - loss: 0.52 - ETA: 17s - loss: 0.53 - ETA: 17s - loss: 0.52 - ETA: 17s - loss: 0.53 - ETA: 17s - loss: 0.53 - ETA: 17s - loss: 0.52 - ETA: 17s - loss: 0.52 - ETA: 16s - loss: 0.52 - ETA: 17s - loss: 0.52 - ETA: 17s - loss: 0.52 - ETA: 17s - loss: 0.52 - ETA: 17s - loss: 0.52 - ETA: 17s - loss: 0.52 - ETA: 16s - loss: 0.52 - ETA: 17s - loss: 

55381/55381 [==============================] - ETA: 0s - loss: 0.507 - 21s 380us/step - loss: 0.5072 - val_loss: 0.4633
Epoch 3/50
55381/55381 [==============================] - ETA: 13s - loss: 0.44 - ETA: 16s - loss: 0.51 - ETA: 19s - loss: 0.50 - ETA: 18s - loss: 0.50 - ETA: 22s - loss: 0.50 - ETA: 24s - loss: 0.50 - ETA: 21s - loss: 0.49 - ETA: 21s - loss: 0.49 - ETA: 19s - loss: 0.49 - ETA: 20s - loss: 0.50 - ETA: 20s - loss: 0.50 - ETA: 22s - loss: 0.50 - ETA: 22s - loss: 0.49 - ETA: 23s - loss: 0.49 - ETA: 22s - loss: 0.49 - ETA: 23s - loss: 0.49 - ETA: 22s - loss: 0.48 - ETA: 22s - loss: 0.49 - ETA: 21s - loss: 0.49 - ETA: 21s - loss: 0.49 - ETA: 21s - loss: 0.48 - ETA: 21s - loss: 0.48 - ETA: 22s - loss: 0.48 - ETA: 21s - loss: 0.48 - ETA: 21s - loss: 0.48 - ETA: 21s - loss: 0.48 - ETA: 21s - loss: 0.48 - ETA: 21s - loss: 0.48 - ETA: 21s - loss: 0.48 - ETA: 21s - loss: 0.48 - ETA: 20s - loss: 0.48 - ETA: 20s - loss: 0.48 - ETA: 20s - loss: 0.48 - ETA: 20s - loss: 0.48 - ETA: 2

55381/55381 [==============================] - ETA: 13s - loss: 0.48 - ETA: 23s - loss: 0.47 - ETA: 22s - loss: 0.46 - ETA: 22s - loss: 0.48 - ETA: 22s - loss: 0.49 - ETA: 20s - loss: 0.49 - ETA: 19s - loss: 0.49 - ETA: 18s - loss: 0.48 - ETA: 19s - loss: 0.48 - ETA: 19s - loss: 0.48 - ETA: 19s - loss: 0.48 - ETA: 18s - loss: 0.47 - ETA: 19s - loss: 0.46 - ETA: 18s - loss: 0.46 - ETA: 18s - loss: 0.46 - ETA: 18s - loss: 0.46 - ETA: 18s - loss: 0.46 - ETA: 18s - loss: 0.46 - ETA: 18s - loss: 0.46 - ETA: 19s - loss: 0.46 - ETA: 19s - loss: 0.46 - ETA: 19s - loss: 0.46 - ETA: 19s - loss: 0.46 - ETA: 19s - loss: 0.46 - ETA: 19s - loss: 0.46 - ETA: 19s - loss: 0.46 - ETA: 18s - loss: 0.46 - ETA: 18s - loss: 0.46 - ETA: 18s - loss: 0.46 - ETA: 18s - loss: 0.46 - ETA: 17s - loss: 0.46 - ETA: 17s - loss: 0.46 - ETA: 17s - loss: 0.46 - ETA: 17s - loss: 0.46 - ETA: 17s - loss: 0.46 - ETA: 17s - loss: 0.46 - ETA: 17s - loss: 0.46 - ETA: 17s - loss: 0.46 - ETA: 17s - loss: 0.46 - ETA: 17s - loss: 

54784/55381 [============================>.] - ETA: 13s - loss: 0.45 - ETA: 20s - loss: 0.49 - ETA: 23s - loss: 0.46 - ETA: 18s - loss: 0.44 - ETA: 19s - loss: 0.45 - ETA: 21s - loss: 0.45 - ETA: 20s - loss: 0.45 - ETA: 20s - loss: 0.46 - ETA: 21s - loss: 0.46 - ETA: 20s - loss: 0.46 - ETA: 21s - loss: 0.45 - ETA: 21s - loss: 0.45 - ETA: 20s - loss: 0.45 - ETA: 19s - loss: 0.45 - ETA: 20s - loss: 0.45 - ETA: 20s - loss: 0.45 - ETA: 20s - loss: 0.45 - ETA: 20s - loss: 0.44 - ETA: 20s - loss: 0.44 - ETA: 20s - loss: 0.44 - ETA: 20s - loss: 0.44 - ETA: 19s - loss: 0.44 - ETA: 19s - loss: 0.44 - ETA: 19s - loss: 0.44 - ETA: 19s - loss: 0.44 - ETA: 20s - loss: 0.44 - ETA: 20s - loss: 0.44 - ETA: 20s - loss: 0.44 - ETA: 20s - loss: 0.44 - ETA: 20s - loss: 0.44 - ETA: 20s - loss: 0.44 - ETA: 19s - loss: 0.44 - ETA: 20s - loss: 0.44 - ETA: 20s - loss: 0.44 - ETA: 20s - loss: 0.44 - ETA: 19s - loss: 0.44 - ETA: 20s - loss: 0.44 - ETA: 19s - loss: 0.44 - ETA: 19s - loss: 0.44 - ETA: 19s - loss: 

...

55381/55381 [==============================] - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 0s - loss: 0.377 - ETA: 0s - loss: 0.377 - ETA: 0s - loss: 0.377 - ETA: 0s - loss: 0.377 - ETA: 0s - loss: 0.377 - ETA: 0s - loss: 0.377 - ETA: 0s - loss: 0.377 - ETA: 0s - loss: 0.377 - ETA: 0s - loss: 0.377 - ETA: 0s - loss: 0.377 - ETA: 0s - loss: 0.377 - ETA: 0s - loss: 0.377 - ETA: 0s - loss: 0.377 - ETA: 0s - loss: 0.377 - ETA: 0s - loss: 0.377 - ETA: 0s - loss: 0.377 - 23s 415us/step - loss: 0.3772 - val_loss: 0.3778
Epoch 47/50
48896/55381 [=========================>....] - ETA: 13s - loss: 0.35 - ETA: 23s - loss: 0.40 - ETA: 31s - loss: 0.38 - ETA: 33s - loss: 0.36 - ETA: 27s - loss: 0.37 - ETA: 27s - loss: 0.36 - ETA: 24s - loss: 0.35 - ETA: 25s - loss: 0.35 - ETA: 24s - loss: 0.37 - ETA: 26s - loss: 0.37 - ETA: 25s - loss: 0.37 - ETA: 24s - loss: 0.36 - ETA: 24s - loss: 0.36 - ETA: 

50240/55381 [==========================>...] - ETA: 13s - loss: 0.29 - ETA: 29s - loss: 0.38 - ETA: 27s - loss: 0.35 - ETA: 27s - loss: 0.35 - ETA: 24s - loss: 0.37 - ETA: 23s - loss: 0.37 - ETA: 22s - loss: 0.37 - ETA: 21s - loss: 0.36 - ETA: 20s - loss: 0.36 - ETA: 20s - loss: 0.37 - ETA: 20s - loss: 0.37 - ETA: 20s - loss: 0.37 - ETA: 21s - loss: 0.37 - ETA: 21s - loss: 0.37 - ETA: 21s - loss: 0.36 - ETA: 22s - loss: 0.37 - ETA: 22s - loss: 0.37 - ETA: 22s - loss: 0.37 - ETA: 22s - loss: 0.37 - ETA: 22s - loss: 0.36 - ETA: 22s - loss: 0.36 - ETA: 21s - loss: 0.36 - ETA: 21s - loss: 0.36 - ETA: 21s - loss: 0.36 - ETA: 21s - loss: 0.36 - ETA: 21s - loss: 0.36 - ETA: 21s - loss: 0.36 - ETA: 21s - loss: 0.37 - ETA: 21s - loss: 0.36 - ETA: 21s - loss: 0.37 - ETA: 21s - loss: 0.36 - ETA: 22s - loss: 0.36 - ETA: 22s - loss: 0.36 - ETA: 22s - loss: 0.36 - ETA: 22s - loss: 0.36 - ETA: 22s - loss: 0.36 - ETA: 21s - loss: 0.36 - ETA: 21s - loss: 0.36 - ETA: 21s - loss: 0.36 - ETA: 21s - loss: 

55381/55381 [==============================] - ETA: 2s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 1s - loss: 0.378 - ETA: 0s - loss: 0.378 - ETA: 0s - loss: 0.378 - ETA: 0s - loss: 0.378 - ETA: 0s - loss: 0.378 - ETA: 0s - loss: 0.378 - ETA: 0s - loss: 0.378 - ETA: 0s - loss: 0.378 - ETA: 0s - loss: 0.378 - ETA: 0s - loss: 0.378 - ETA: 0s - loss: 0.378 - ETA: 0s - loss: 0.378 - ETA: 0s - loss: 0.378 - ETA: 0s - loss: 0.378 - ETA: 0s - loss: 0.378 - ETA: 0s - loss: 0.378 - ETA: 0s - loss: 0.378 - ETA: 0s - loss: 0.378 - ETA: 0s - loss: 0.378 - ETA: 0s - loss: 0.378 - 23s 422us/step - loss: 0.3783 - val_loss: 0.3753
Epoch 49/50
4601

51968/55381 [===========================>..] - ETA: 27s - loss: 0.33 - ETA: 49s - loss: 0.38 - ETA: 51s - loss: 0.40 - ETA: 40s - loss: 0.39 - ETA: 41s - loss: 0.38 - ETA: 37s - loss: 0.37 - ETA: 32s - loss: 0.38 - ETA: 30s - loss: 0.38 - ETA: 27s - loss: 0.39 - ETA: 26s - loss: 0.38 - ETA: 24s - loss: 0.39 - ETA: 23s - loss: 0.37 - ETA: 23s - loss: 0.37 - ETA: 24s - loss: 0.38 - ETA: 23s - loss: 0.37 - ETA: 22s - loss: 0.38 - ETA: 23s - loss: 0.38 - ETA: 23s - loss: 0.38 - ETA: 23s - loss: 0.38 - ETA: 24s - loss: 0.38 - ETA: 24s - loss: 0.37 - ETA: 24s - loss: 0.37 - ETA: 24s - loss: 0.37 - ETA: 24s - loss: 0.37 - ETA: 24s - loss: 0.37 - ETA: 24s - loss: 0.37 - ETA: 24s - loss: 0.37 - ETA: 23s - loss: 0.37 - ETA: 23s - loss: 0.37 - ETA: 23s - loss: 0.37 - ETA: 23s - loss: 0.37 - ETA: 23s - loss: 0.37 - ETA: 23s - loss: 0.37 - ETA: 23s - loss: 0.37 - ETA: 23s - loss: 0.37 - ETA: 22s - loss: 0.37 - ETA: 22s - loss: 0.37 - ETA: 22s - loss: 0.37 - ETA: 22s - loss: 0.37 - ETA: 22s - loss: 

55381/55381 [==============================] - ETA: 1s - loss: 0.374 - ETA: 1s - loss: 0.374 - ETA: 1s - loss: 0.374 - ETA: 1s - loss: 0.374 - ETA: 1s - loss: 0.374 - ETA: 1s - loss: 0.374 - ETA: 0s - loss: 0.374 - ETA: 0s - loss: 0.374 - ETA: 0s - loss: 0.374 - ETA: 0s - loss: 0.375 - ETA: 0s - loss: 0.374 - ETA: 0s - loss: 0.375 - ETA: 0s - loss: 0.374 - ETA: 0s - loss: 0.375 - ETA: 0s - loss: 0.375 - ETA: 0s - loss: 0.375 - ETA: 0s - loss: 0.375 - ETA: 0s - loss: 0.375 - ETA: 0s - loss: 0.375 - ETA: 0s - loss: 0.375 - ETA: 0s - loss: 0.375 - 22s 404us/step - loss: 0.3754 - val_loss: 0.3790


In [52]:
predictions = model.predict(xvalid_glove_scl)

In [53]:
predict_label = [np.argmax(i) for i in predictions]

In [54]:
predict_label = lbl_enc.inverse_transform(predict_label)

C:\Users\narendran.thesma\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [55]:
yvalid_label = lbl_enc.inverse_transform(yvalid)

C:\Users\narendran.thesma\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [56]:
test_df = pd.DataFrame({'description' : xvalid, 'actuals' : yvalid_label, 'predictions' : predict_label})

In [57]:
test_df.head()

,description,actuals,predictions
0,nike japan ナイキ エア ヴェイパーマックス フライニット 2 メンズ ランニング...,performance,performance
1,puma japan golf ゴルフw ナカワタパンツ puma｜ゴルフw ナカワタパンツ...,performance,performance
2,puma japan archive ストラクト クルースウェット ウィメンズサイズのクルー...,sport inspired,sport inspired
3,adidas japan オリジナルス トラックトップジャージ［eqt track top ...,sport inspired,sport inspired
4,adidas japan タンゴ ソックス 1978年fifaワールドカップアルゼンチン大会...,performance,performance


In [58]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [59]:
f1_score(test_df.actuals, test_df.predictions,average="macro")

0.8271767728174146

In [60]:
accuracy_score(test_df.actuals, test_df.predictions)

0.8323041923951902

In [141]:
chk_token = [' '.join(x) for x in tokenized_text]

In [145]:
chk_token[2]

'adidas   ピュアブーストエックス   【 pure   boost   x   tr 】    ,    サイズ ：   素材 ：   生産国 ：'

In [78]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 300

token.fit_on_texts(list(xtrain.description) + list(xvalid.description))
xtrain_seq = token.texts_to_sequences(xtrain.description)
xvalid_seq = token.texts_to_sequences(xvalid.description)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [146]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 300

token.fit_on_texts(chk_token)
xtrain_seq = token.texts_to_sequences(chk_token)
xvalid_seq = token.texts_to_sequences(xvalid.description)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [151]:
xtrain_pad[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [61]:
#token.word_index

In [63]:
#token.word_docs

In [80]:
#(word_index)

In [96]:
# create an embedding matrix for the words we have in the dataset
#embeddings_index = {}
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|█████████████████████████████████████████████████████████████████████████| 72374/72374 [00:01<00:00, 37525.51it/s]


In [97]:
embedding_matrix.shape

(72375, 300)

In [98]:
len(word_index)

72374

In [95]:
#word_index.items()

In [124]:
# Imp
# final_set = set()
# for i in range(len(tokenized_text)):
#     temp = set(tokenized_text[i])
#     final_set = final_set.union(temp)

In [127]:
# final_list = list(final_set)

In [114]:
tokenized_text[1]

['unknown']

In [130]:
#word_model.wv[final_list[0]]

In [131]:
# create an embedding matrix for the words we have in the dataset
#embeddings_index = {}
# embedding_matrix = np.zeros((len(final_list) + 1, 300))
# for i in range(len(final_list)):
#     embedding_vector = word_model.wv[final_list[i]]
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector

In [35]:
xtrain_pad

array([[     0,      0,      0, ...,      2,     74,  60989],
       [     0,      0,      0, ...,      0,      0,     58],
       [     0,      0,      0, ...,      6,     19,  60990],
       ...,
       [     0,      0,      0, ..., 120361,     37,  55592],
       [     0,      0,      0, ...,  14712,     26,    103],
       [     0,      0,      0, ...,      0,      0,     58]])

In [36]:
#xtrain_pad[0]

In [135]:
from keras import optimizers

# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(2))
model.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.00001, decay=1e-5, momentum=0.8, nesterov=True)
rmsprop = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer= rmsprop)

In [138]:
model.fit(xtrain_pad, y=ytrain_enc, batch_size=256, epochs=2, verbose=1, validation_data=(xvalid_pad, yvalid_enc))

Train on 55381 samples, validate on 6154 samples
Epoch 1/2


InvalidArgumentError: indices[116,297] = 39038 is not in [0, 36284)
	 [[Node: embedding_2/embedding_lookup = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@spatial_dropout1d_1/cond/Switch_1"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_2/embeddings/read, embedding_2/Cast, embedding_2/embedding_lookup/axis)]]

In [41]:
predictions = model.predict(xvalid_pad)

In [42]:
predict_label = [np.argmax(i) for i in predictions]

In [43]:
predict_label = lbl_enc.inverse_transform(predict_label)

C:\Users\narendran.thesma\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [44]:
yvalid_label = lbl_enc.inverse_transform(yvalid)

C:\Users\narendran.thesma\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [47]:
predict_label

array(['performance', 'sport inspired', 'sport inspired', ...,
       'performance', 'performance', 'sport inspired'], dtype=object)

In [48]:
test_df = pd.DataFrame({'description' : xvalid.description, 'actuals' : yvalid_label, 'predictions' : predict_label})

In [49]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [50]:
f1_score(test_df.actuals, test_df.predictions,average="macro")

0.7415497130750217

In [51]:
accuracy_score(test_df.actuals, test_df.predictions)

0.745206369840754